In [1]:
from pathlib import Path
from ast import literal_eval
from collections import Counter
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
# to do
# find best features for each fold (count of each)
# plot cumulatif histogramm
# find best beatures for each fold
# do the intersection

# Load data

In [3]:
crossval = {}
nb_cv = 5

In [4]:
for i in range(nb_cv):
    # load logs from brute force
    file_name = f'best_results_fold_{i}.log'
    data_one_cv = pd.read_csv(Path('..') / 'results' / 'cross_val' / file_name, sep='|', header=0, names=['mean', 'std', 'scores_cv', 'features'])
    data_one_cv.sort_values('mean', inplace=True, ascending=False)
    data_one_cv.reset_index(drop=True, inplace=True)
    crossval[i] = {'logs': data_one_cv}
    
    # load feature and labels
    crossval[i]['X_train'] = pickle.load(open(Path('..') / 'data' / 'for_training' / f'X_train_{i}.pkl', 'rb')).reset_index(drop=True)
    crossval[i]['X_test'] = pickle.load(open(Path('..') / 'data' / 'for_training' / f'X_test_{i}.pkl', 'rb')).reset_index(drop=True)
    crossval[i]['y_train'] = pickle.load(open(Path('..') / 'data' / 'for_training' / f'y_train_{i}.pkl', 'rb')).reset_index(drop=True)
    crossval[i]['y_test'] = pickle.load(open(Path('..') / 'data' / 'for_training' / f'y_test_{i}.pkl', 'rb')).reset_index(drop=True)


In [5]:
crossval[0]['logs'].head(100)

,mean,std,scores_cv,features
0,77.7,6.4,[0.77272727 0.86363636 0.77272727 0.80952381 0...,"['7:63000001-64000000_zscore', '9:88000001-890..."
1,76.9,3.7,[0.72727273 0.77272727 0.72727273 0.80952381 0...,"['7:34000001-35000000_zscore', '13:109000001-1..."
2,76.8,3.3,[0.77272727 0.77272727 0.81818182 0.76190476 0...,"['8:11000001-12000000_zscore', '12:68000001-69..."
3,76.8,9.1,[0.72727273 0.77272727 0.86363636 0.85714286 0...,"['3:155000001-156000000_zscore', '17:56000001-..."
4,76.7,6.4,[0.81818182 0.81818182 0.81818182 0.66666667 0...,"['17:68000001-69000000_zscore', '7:94000001-95..."
...,...,...,...,...
95,72.2,7.9,[0.63636364 0.81818182 0.77272727 0.76190476 0...,"['13:44000001-45000000_zscore', '15:96000001-9..."
96,72.2,6.5,[0.72727273 0.63636364 0.81818182 0.76190476 0...,"['10:115000001-116000000_zscore', '21:19000001..."
97,72.2,5.3,[0.72727273 0.68181818 0.81818182 0.66666667 0...,"['3:77000001-78000000_zscore', '4:13000001-140..."
98,72.2,7.8,[0.68181818 0.68181818 0.81818182 0.80952381 0...,"['3:35000001-36000000_zscore', '7:23000001-240..."


# Find commun features

In [6]:
nb_experiments = 500
nb_features = 200

for i in range(nb_cv):
    features_list = [literal_eval(x) for x in crossval[i]['logs'].loc[:nb_experiments, 'features'].to_list()]
    features_flat_list = [item for sublist in features_list for item in sublist]
    
    c = Counter(features_flat_list)
    features_name = [x[0] for x in c.most_common()[:nb_features]]
    crossval[i]['features_count'] = c.most_common()[:nb_features]
    crossval[i]['features_name'] = features_name 

# Find best features
## Train models

In [7]:
def model_manual_train_test(X_train, y_train, X_test, y_test):
    lr = LogisticRegression(C=0.01, class_weight='balanced', random_state=5)
    ss = StandardScaler().fit(X_train)
    X_train_ss = ss.transform(X_train)
    lr.fit(X_train_ss, y_train)
    y_train_pred = lr.predict(X_train_ss)

    train_acc = accuracy_score(y_train_pred, y_train)
    train_scores = precision_recall_fscore_support(y_train, y_train_pred, labels=[0, 1])

    X_test_ss = ss.transform(X_test)
    y_test_pred = lr.predict(X_test_ss)

    test_acc = accuracy_score(y_test_pred, y_test)
    test_scores = precision_recall_fscore_support(y_test, y_test_pred, labels=[0, 1])
    
    return train_acc, train_scores, test_acc, test_scores

In [9]:
nb_features = range(50, 200)
res = {}

for cv in range(nb_cv):
    for nb_experiments in tqdm(range(500, 1000, 50)):

        features_list = [literal_eval(x) for x in crossval[cv]['logs'].loc[:nb_experiments, 'features'].to_list()]
        features_flat_list = [item for sublist in features_list for item in sublist]

        c = Counter(features_flat_list)

        for nb_feat in nb_features:
            features_name = [x[0] for x in c.most_common()[:nb_feat]]
            features_name = [x.replace('ratio', 'zscore') for x in features_name]

            X_train_light = crossval[cv][f'X_train'].loc[:, features_name]
            X_test_light = crossval[cv][f'X_test'].loc[:, features_name]

            train_acc, train_scores, test_acc, test_scores = model_manual_train_test(X_train_light, crossval[cv]['y_train'].tolist(),
                                                                                     X_test_light, crossval[cv]['y_test'].tolist())
                                                                                     
            res[f'{cv}-{nb_feat}-{nb_experiments}'] = (train_acc, train_scores, test_acc, test_scores, features_name)


100%|██████████████████████████████████████████████████| 10/10 [00:23<00:00,  2.35s/it]


In [10]:
list_train_acc, list_train_scores, list_test_acc, list_test_scores, list_features_name = list(zip(*res.values()))

In [11]:
res_df = pd.DataFrame({'concat_info': res.keys(), 
                       'train_acc': list_train_acc, 'train_scores': list_train_scores,
                       'test_acc': list_test_acc, 'test_scores': list_test_scores,
                       'features_name': list_features_name})

In [12]:
context = pd.DataFrame(res_df['concat_info'].str.split('-').tolist(), columns= ['cv_nb', 'feat_nb', 'exp_nb'])

In [13]:
res_df = pd.concat([context, res_df], axis=1)

In [14]:
res_df.head()

,cv_nb,feat_nb,exp_nb,concat_info,train_acc,train_scores,test_acc,test_scores,features_name
0,0,50,500,0-50-500,0.962963,"([0.9682539682539683, 0.9555555555555556], [0....",0.444444,"([0.5, 0.36363636363636365], [0.53333333333333...","[3:165000001-166000000_zscore, 13:70000001-710..."
1,0,51,500,0-51-500,0.962963,"([0.9682539682539683, 0.9555555555555556], [0....",0.444444,"([0.5, 0.36363636363636365], [0.53333333333333...","[3:165000001-166000000_zscore, 13:70000001-710..."
2,0,52,500,0-52-500,0.962963,"([0.9682539682539683, 0.9555555555555556], [0....",0.444444,"([0.5, 0.36363636363636365], [0.53333333333333...","[3:165000001-166000000_zscore, 13:70000001-710..."
3,0,53,500,0-53-500,0.962963,"([0.9682539682539683, 0.9555555555555556], [0....",0.444444,"([0.5, 0.36363636363636365], [0.53333333333333...","[3:165000001-166000000_zscore, 13:70000001-710..."
4,0,54,500,0-54-500,0.962963,"([0.9682539682539683, 0.9555555555555556], [0....",0.444444,"([0.5, 0.36363636363636365], [0.53333333333333...","[3:165000001-166000000_zscore, 13:70000001-710..."


## Results analysis

In [18]:
for i in range(nb_cv):
    rest_df_one_cv = res_df[res_df['cv_nb'] == str(i)]
    print(rest_df_one_cv[['train_acc', 'test_acc']].sort_values('test_acc', ascending=False))
    crossval[i]['best_features_name'] = rest_df_one_cv[['train_acc', 'test_acc', 'features_name']].sort_values('test_acc').tail(1)['features_name'].tolist()

     train_acc  test_acc
686   1.000000  0.592593
614   0.972222  0.592593
360   1.000000  0.592593
366   1.000000  0.592593
367   1.000000  0.592593
..         ...       ...
471   0.990741  0.444444
473   0.990741  0.444444
157   0.962963  0.407407
156   0.962963  0.407407
158   0.962963  0.407407

[1500 rows x 2 columns]
      train_acc  test_acc
2410   0.972222  0.444444
2405   0.962963  0.444444
2849   1.000000  0.444444
2855   0.972222  0.444444
2848   1.000000  0.444444
...         ...       ...
2139   0.981481  0.259259
1574   1.000000  0.259259
2142   0.990741  0.259259
2143   1.000000  0.259259
1575   1.000000  0.259259

[1500 rows x 2 columns]
      train_acc  test_acc
3292   1.000000  0.703704
3293   1.000000  0.703704
3599   1.000000  0.703704
3242   0.990741  0.666667
4354   0.972222  0.666667
...         ...       ...
3627   0.981481  0.444444
3043   0.981481  0.444444
3042   0.981481  0.444444
3041   0.990741  0.444444
3040   0.981481  0.444444

[1500 rows x 2 columns]
 

In [24]:
res_df.loc[res_df['cv_nb'] == '2', ['train_acc', 'test_acc', 'feat_nb', 'exp_nb', 'features_name']].sort_values('test_acc', ascending=False).head(20)

,train_acc,test_acc,feat_nb,exp_nb,features_name
3292,1.000000,0.703704,192,550,"[11:10000001-11000000_zscore, 4:104000001-1050..."
3293,1.000000,0.703704,193,550,"[11:10000001-11000000_zscore, 4:104000001-1050..."
3599,1.000000,0.703704,199,650,"[11:10000001-11000000_zscore, 4:104000001-1050..."
3242,0.990741,0.666667,142,550,"[11:10000001-11000000_zscore, 4:104000001-1050..."
4354,0.972222,0.666667,54,950,"[11:10000001-11000000_zscore, 17:58000001-5900..."
3083,0.990741,0.666667,133,500,"[11:10000001-11000000_zscore, 3:165000001-1660..."
3082,0.990741,0.666667,132,500,"[11:10000001-11000000_zscore, 3:165000001-1660..."
3081,0.990741,0.666667,131,500,"[11:10000001-11000000_zscore, 3:165000001-1660..."
3080,0.990741,0.666667,130,500,"[11:10000001-11000000_zscore, 3:165000001-1660..."
3597,1.000000,0.666667,197,650,"[11:10000001-11000000_zscore, 4:104000001-1050..."


In [19]:
features_cv_0 = res_df.loc[(res_df['cv_nb'] == '0') & (res_df['feat_nb'] == '64' ) & (res_df['exp_nb'] == '700'), 'features_name'].tolist()[0]
features_cv_1 = res_df.loc[(res_df['cv_nb'] == '1') & (res_df['feat_nb'] == '55' ) & (res_df['exp_nb'] == '800'), 'features_name'].tolist()[0]
features_cv_2 = res_df.loc[(res_df['cv_nb'] == '2') & (res_df['feat_nb'] == '52' ) & (res_df['exp_nb'] == '950'), 'features_name'].tolist()[0]
features_cv_3 = res_df.loc[(res_df['cv_nb'] == '3') & (res_df['feat_nb'] == '93' ) & (res_df['exp_nb'] == '950'), 'features_name'].tolist()[0]
features_cv_4 = res_df.loc[(res_df['cv_nb'] == '4') & (res_df['feat_nb'] == '56' ) & (res_df['exp_nb'] == '600'), 'features_name'].tolist()[0]

In [20]:
best_features_all_cv = features_cv_0 + features_cv_1 + features_cv_2 + features_cv_3 + features_cv_4

In [21]:
from collections import Counter
c = Counter(best_features_all_cv)
c.most_common()

[('3:165000001-166000000_zscore', 5),
 ('1:109000001-110000000_zscore', 5),
 ('12:72000001-73000000_zscore', 5),
 ('22:44000001-45000000_zscore', 5),
 ('13:70000001-71000000_zscore', 4),
 ('19:43000001-44000000_zscore', 4),
 ('6:33000001-34000000_zscore', 4),
 ('1:187000001-188000000_zscore', 4),
 ('14:37000001-38000000_zscore', 4),
 ('5:156000001-157000000_zscore', 4),
 ('11:10000001-11000000_zscore', 4),
 ('12:127000001-128000000_zscore', 4),
 ('1:217000001-218000000_zscore', 3),
 ('7:49000001-50000000_zscore', 3),
 ('3:163000001-164000000_zscore', 3),
 ('5:121000001-122000000_zscore', 3),
 ('2:150000001-151000000_zscore', 3),
 ('7:26000001-27000000_zscore', 3),
 ('1:121000001-122000000_zscore', 3),
 ('1:22000001-23000000_zscore', 3),
 ('2:160000001-161000000_zscore', 3),
 ('14:29000001-30000000_zscore', 3),
 ('4:122000001-123000000_zscore', 3),
 ('15:90000001-91000000_zscore', 3),
 ('7:93000001-94000000_zscore', 3),
 ('14:73000001-74000000_zscore', 3),
 ('9:11000001-12000000_zscore'